In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
#automodel bir seq-2-seq model yükler 
#autotokenizer modelin eğitiminde ve çıkarımında kullanılan tokenizasyon işlemlerini yapar
#genarition_config modelin üretirken parametrelerini ayarlamaya yarar
#modelin eğitimine ilişkin tüm argümanları (örneğin, öğrenme oranı, batch boyutu, epoch sayısı gibi) kapsayan bir yapı sunar.
#modelin eğitimi ve değerlendirilmesi için yüksek seviyeli bir API sunar
import torch
import time
import evaluate 
import pandas as pd
import numpy as np


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/data/metrics/__init__.py:19: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.stats import pearsonr, spearmanr


In [2]:
hugging_face_dataset_name="knkarthick/dialogsum"
dataset = load_dataset(hugging_face_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [3]:
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype =torch.bfloat16) # bu herhangi bir fine-tuning yapılmayan model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()

    percentage_trainable = (trainable_model_params / all_model_params) * 100 if all_model_params > 0 else 0
    
    return (f"trainable model parameters: {trainable_model_params}\n"
            f"all model parameters: {all_model_params}\n"
            f"percentage of trainable model parameters: {percentage_trainable:.2f}%")

print(print_number_of_trainable_model_parameters(original_model))


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [5]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.
{dialogue}
Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}') # verdiğimiz input bu şekilde
print(dash_line)
print(f'PRINT BASELINE SUMMARY:\n{summary}') # asıl olması gereken özet bu şekilde 
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}') #modelin ürettiği özet bu şekilde


      

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
Summary:

---------------------------------------------------------------------

In [6]:
dataset.column_names

{'train': ['id', 'dialogue', 'summary', 'topic'],
 'validation': ['id', 'dialogue', 'summary', 'topic'],
 'test': ['id', 'dialogue', 'summary', 'topic']}

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation . \n\n'
    end_prompt = '\n\nSummary:'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example ["dialogue"]]
    example["input_ids"] = tokenizer(prompt,padding="max_length",truncation=True,return_tensors="pt").input_ids
    example['labels']= tokenizer(example["summary"],padding = "max_length",truncation=True,return_tensors="pt").input_ids
    
    return example

tokenized_datasets = dataset.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary',])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index%100 ==0, with_indices=True)
#veri kümesinin boyutunu küçültmek içni sadece 100 ile bölümünden 0 kalanları alıyoruz.

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)



Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


In [11]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay = 0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer.train()

Step,Training Loss
1,47.750000


TrainOutput(global_step=1, training_loss=47.75, metrics={'train_runtime': 74.7284, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.013, 'total_flos': 5478058819584.0, 'train_loss': 47.75, 'epoch': 0.0625})

In [16]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [17]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint

fatal error: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


In [14]:
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin


ls: cannot access ./flan-dialogue-summary-checkpoint/pytorch_model.bin: No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint",torch_dtype=torch.bfloat16)

ValueError: Unrecognized model in ./flan-dialogue-summary-checkpoint. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, fsmt, fun